# Ejercicio 7

Dadas las dimensiones y la discretización, se plantea la solución a través de una matriz de 200x200 que evoluciona en instantes discretos, cada uno asociado a un estado estático a partir del cual se determina el próximo. Cada una de las celdas puede estar ocupada por un agente, cuya posición inicial se determina de forma aleatoria (con igual probabilidad de ocupar cualquier celda, y utilizando recálculo en el caso de colisión con un agente ya asignado). 

Para simular el contagio, a partir de un estado de la matriz se recorren las celdas adyacentes a cada agente infectado hasta la 6ta celda de distancia. Para cada celda ocupada por un agente no enfermo a una determinada distancia, se utiliza una variable aleatoria entre 0 y 1 para determinar si el mismo se contagia, de acuerdo a las probabilidades dadas en la consigna. 

Se guardan los respectivos estados de cada iteración en términos de arreglos de agentes enfermos y agentes sanos para poder realizar la animación.


In [1]:
import random 
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator, FormatStrFormatter)

In [2]:
# Default for numpy: edgeitems = 3, line_width = 80
np.set_printoptions(edgeitems = 14, linewidth = 300)

In [3]:
STATE_LEN = 200 
ELEMENTS = STATE_LEN * STATE_LEN

AGENTS = 1000
AGENTS_A = int(AGENTS * 0.7)
AGENTS_B = int(AGENTS * 0.25)
AGENTS_C = int(AGENTS * 0.05)
AGENTS_INF = int(AGENTS * 0.02)

INFECTED = "_X"
AGENT_A = "A"
AGENT_B = "B"
AGENT_C = "C"
AGENT_KINDS = [AGENT_A, AGENT_B, AGENT_C]
EMPTY = "."
ZERO_TO_STATE_LEN = range(STATE_LEN)

INFECT_NEAR_P = 0.7
INFECT_FAR_P = 0.5

DIRECTIONS = [(0, 1), (0, -1),
              (1, 0), (-1, 0)]

In [4]:
to_infect = np.random.choice( range(AGENTS), AGENTS_INF )
infected_pos = []
healthy_pos = []

In [5]:
def setAgents(agents_len, agent, asigned):
    for i in range(agents_len):
        while True:
            x = np.random.choice(ZERO_TO_STATE_LEN, 1)[0]
            y = np.random.choice(ZERO_TO_STATE_LEN, 1)[0]
            
            if ( state[x][y] == EMPTY ):
                break

        state[x][y] = str(asigned) + "_" + agent
        
        if (asigned in to_infect):
            state[x][y] += INFECTED
            infected_pos.append( (asigned, x, y) )
        
        else:
            healthy_pos.append( (asigned, x, y) )
            
        asigned += 1
    
    return asigned

In [6]:
state = [] 

# State initialization
for i in ZERO_TO_STATE_LEN:
    state.append([])
    for j in ZERO_TO_STATE_LEN:
        state[i].append(EMPTY)

asigned = 0
asigned = setAgents(AGENTS_A, AGENT_A, asigned)
asigned = setAgents(AGENTS_B, AGENT_B, asigned)
asigned = setAgents(AGENTS_C, AGENT_C, asigned)

# print( np.matrix(state) ) 

In [7]:
print( str( len( infected_pos ) ) + " " + str( len( healthy_pos ) ) )

20 980


In [8]:
def isPossibleMovement(state, x, y):
    if x<0 or y<0:
        return False
    try:
        state[x][y]
        return True
    except IndexError:
        return False

In [9]:
def _infect(state, elements, x, y, prob):
    for x, y in elements:
        if ( not isPossibleMovement(state, x, y) ) or (state[x][y] ==  EMPTY):
            continue
        
        agent_kind = state[x][y].split("_")[1]
        
        try:
            state[x][y].split("_")[2]
            continue
        except IndexError:
            pass
            
        if (random.random() < prob):
            state[x][y] += INFECTED

In [10]:
def infect(state, next_state, x, y):
    near = [
        # dist 1
        (x, y + 1), (x + 1, y + 1), (x + 1, y), (x + 1, y - 1), 
        (x, y - 1), (x - 1, y - 1), (x - 1, y), (x - 1, y + 1), 
    
        # dist = 2
        (x, y + 2), (x + 2, y), (x, y - 2), (x - 2, y), 
        (x + 1, y + 1), (x + 1, y - 1), (x - 1, y - 1), (x - 1, y + 1),
    
        # dist = 3
        (x, y + 3), (x + 3, y), (x, y - 3), (x - 3, y),
        (x + 1, y + 2), (x + 2, y + 1), (x + 2, y - 1), (x + 1, y - 2),
        (x - 1, y - 2), (x - 2, y - 1), (x - 2, y + 1), (x - 1, y + 2),
    ]
    
    _infect(state, next_state, near, x, y, INFECT_NEAR_P)
        
    far = [
        # dist 4
        (x, y + 4), (x + 4, y), (x, y - 4), (x - 4, y),
        (x + 1, y + 3), (x + 1, y - 3), (x - 1, y - 3), (x - 1, y + 3),
        (x + 2, y + 2), (x + 2, y - 2), (x - 2, y - 2), (x - 2, y + 2),
        (x + 3, y + 1), (x + 3, y - 1), (x - 3, y - 1), (x - 3, y + 1),
        
        # dist 5
        (x, y + 5), (x + 5, y), (x, y - 5), (x - 5, y),
        (x + 1, y + 4), (x + 1, y - 4), (x - 1, y - 4), (x - 1, y + 4),
        (x + 2, y + 3), (x + 2, y - 3), (x - 2, y - 3), (x - 2, y + 3),
        (x + 3, y + 2), (x + 3, y - 2), (x - 3, y - 2), (x - 3, y + 2),
        (x + 4, y + 1), (x + 4, y - 1), (x - 4, y - 1), (x - 3, y + 1),
        
        # dist 6
        (x, y + 6), (x + 6, y), (x, y - 6), (x - 6, y),
        (x + 1, y + 5), (x + 1, y - 5), (x - 1, y - 5), (x - 1, y + 5),
        (x + 2, y + 4), (x + 2, y - 4), (x - 2, y - 4), (x - 2, y + 4),
        (x + 3, y + 3), (x + 3, y - 3), (x - 3, y - 3), (x - 3, y + 3),
        (x + 4, y + 2), (x + 4, y - 2), (x - 4, y - 2), (x - 4, y + 2),
        (x + 5, y + 1), (x + 5, y - 1), (x - 5, y - 1), (x - 5, y + 1),
    ]
        
    _infect(state, next_state, far, x, y, INFECT_FAR_P)

In [11]:
def notPossibleMovement(state, x, y):
    for dx, dy in DIRECTIONS:
        if not isPossibleMovement(state, x + dx, y + dy):
            continue
            
        if state[x + dx][y + dy] == EMPTY:
            return False
    
    return True

In [12]:
infected_pos_plot = []
healthy_pos_plot = []

infected_pos_plot.append( infected_pos )
healthy_pos_plot.append( healthy_pos ) 

In [13]:
def simulatePandemic(iterations, state):
    for a in range(iterations):
        next_state = np.array( state ).tolist()        

#infect loop
        for i in ZERO_TO_STATE_LEN:
            for j in ZERO_TO_STATE_LEN:
                if state[i][j] == EMPTY:
                    continue
                if INFECTED in state[i][j]:

#moveloop
        for i in ZERO_TO_STATE_LEN:
            for j in ZERO_TO_STATE_LEN:
                if state[i][j] == EMPTY:
                    continue

                
                to_move = False
                
                if AGENT_A in state[i][j]:
                    to_move = True
                
                elif (AGENT_B in state[i][j]) and (a % 2 == 0):
                    to_move = True
                
                elif AGENT_C in state[i][j] and (a % 4 == 0):
                    to_move = True
                    
                if notPossibleMovement(next_state, i, j) or (not to_move):
                    continue 
                    
                dx = 0
                dy = 0
                #Edge case: check with next_state, not with state.
                while next_state[i + dx][j + dy] != EMPTY:
                    dir_i = np.random.choice( range( len(DIRECTIONS) ), 1 )[0]
                    dx, dy = DIRECTIONS[dir_i]
                    
                    if not isPossibleMovement(state, i + dx, j + dy):
                        dx = 0
                        dy = 0
                        continue
                        
                next_state[i][j] = EMPTY
                next_state[i + dx][j + dy] = state[i][j]
           
        
        infected = []
        healthy = []
#recaploop
        for i in ZERO_TO_STATE_LEN:
            for j in ZERO_TO_STATE_LEN:
                if next_state[i][j] == EMPTY:
                    continue

                agent_id = next_state[i][j].split("_")[0]
                    
                if INFECTED in next_state[i][j]:
                    infected.append( (agent_id, i, j) )
            
                else:
                    healthy.append( (agent_id, i, j) )
            
        infected_pos_plot.append(infected)
        healthy_pos_plot.append(healthy) 
       
        state = np.array( next_state ).tolist()       
        
        if len(infected) == AGENTS:
            return
     

In [14]:
simulatePandemic(10, state)

TypeError: infect() missing 1 required positional argument: 'y'

In [ ]:
infected_pos_plot = []
healthy_pos_plot = []

infected_pos_plot.append( infected_pos )
healthy_pos_plot.append( healthy_pos ) 